In [1]:
import pyspark.sql.functions as f

from pyspark.sql.functions import col, size, split, trim, when

In [2]:
#df.unpersist()

In [3]:
# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df2 = spark.sql("select * from default.home_and_kitchen_5_small")
df3 = spark.sql("select * from default.books_5_small")

df = df1.union(df2).union(df3)


In [4]:
test_df = spark.sql("select * from default.reviews_kaggle")

In [5]:
# Convert Unix timestamp to readable date

from pyspark.sql.functions import from_unixtime, to_date

df = df.withColumn("reviewTime", to_date(from_unixtime(df.unixReviewTime))) \
                                                .drop("unixReviewTime")

test_df = test_df.withColumn("reviewTime", to_date(from_unixtime(test_df.unixReviewTime))) \
                                                .drop("unixReviewTime")

lastDate = test_df.agg({"reviewTime": "max"}).collect()[0][0]
from pyspark.sql.functions import lit

df = df.withColumn('lastDate',lit(lastDate))
df = df.withColumn('reviewAge', f.datediff('lastDate', 'reviewTime'))

test_df = test_df.withColumn('lastDate',lit(lastDate))
test_df = test_df.withColumn('reviewAge', f.datediff('lastDate', 'reviewTime'))

In [6]:
from pyspark.sql.functions import year

df = df.withColumn('reviewYear', year('ReviewTime'))
test_df = test_df.withColumn('reviewYear', year('ReviewTime'))

In [7]:
df = df.filter(col('reviewYear')<2018)
#There are no reviews in 2018 in the test set

In [8]:
#Anything less than 2011 was encoded as 2011
df = df.withColumn('Y2011', when(col('reviewYear')<=2011,1).otherwise(0))
df = df.withColumn('Y2012', when(col('reviewYear')==2012,1).otherwise(0))
df = df.withColumn('Y2013', when(col('reviewYear')==2013,1).otherwise(0))
df = df.withColumn('Y2014', when(col('reviewYear')==2014,1).otherwise(0))
df = df.withColumn('Y2015', when(col('reviewYear')==2015,1).otherwise(0))
df = df.withColumn('Y2016', when(col('reviewYear')==2016,1).otherwise(0))
df = df.withColumn('Y2017', when(col('reviewYear')==2017,1).otherwise(0))
#df = df.withColumn('Y2018', when(col('reviewYear')==2018,1).otherwise(0))
test_df = test_df.withColumn('Y2011', when(col('reviewYear')<=2011,1).otherwise(0))
test_df = test_df.withColumn('Y2012', when(col('reviewYear')==2012,1).otherwise(0))
test_df = test_df.withColumn('Y2013', when(col('reviewYear')==2013,1).otherwise(0))
test_df = test_df.withColumn('Y2014', when(col('reviewYear')==2014,1).otherwise(0))
test_df = test_df.withColumn('Y2015', when(col('reviewYear')==2015,1).otherwise(0))
test_df = test_df.withColumn('Y2016', when(col('reviewYear')==2016,1).otherwise(0))
test_df = test_df.withColumn('Y2017', when(col('reviewYear')==2017,1).otherwise(0))
#test_df = test_df.withColumn('Y2018', when(col('reviewYear')==2018,1).otherwise(0))

df = df.withColumn('Overall_1', when(col('overall')==1,1).otherwise(0))
df = df.withColumn('Overall_2', when(col('overall')==2,1).otherwise(0))
df = df.withColumn('Overall_3', when(col('overall')==3,1).otherwise(0))
df = df.withColumn('Overall_4', when(col('overall')==4,1).otherwise(0))
df = df.withColumn('Overall_5', when(col('overall')==5,1).otherwise(0))

test_df = test_df.withColumn('Overall_1', when(col('overall')==1,1).otherwise(0))
test_df = test_df.withColumn('Overall_2', when(col('overall')==2,1).otherwise(0))
test_df = test_df.withColumn('Overall_3', when(col('overall')==3,1).otherwise(0))
test_df = test_df.withColumn('Overall_4', when(col('overall')==4,1).otherwise(0))
test_df = test_df.withColumn('Overall_5', when(col('overall')==5,1).otherwise(0))


In [9]:
df = df.withColumn('reviewYear', when(col('reviewYear')<=2011,2011).otherwise(col('reviewYear')))

In [10]:
6#df = df.filter(col('reviewYear')>=2012)
#df = df.sample(False, 0.25, seed=47)
#df = df.sample(False, 0.10, seed=47)
#df = df.sampleBy("reviewYear", fractions={2011: 0.012, 2012: 0.009, 2013: 0.026, 2014: 0.046, 2015: 0.060, 2016: 0.077, 2017: 0.070}, seed=47)
#df = df.sampleBy("reviewYear", fractions={2011: 0.070, 2012: 0.172, 2013: 0.215, 2014: 0.264, 2015: 0.305, 2016: 0.481, 2017: 0.672}, seed=47)
df = df.sampleBy("reviewYear", fractions={2011: 0.105, 2012: 0.256, 2013: 0.320, 2014: 0.392, 2015: 0.453, 2016: 0.715, 2017: 1.000}, seed=47)
df = df.cache()

print((df.count(), len(df.columns)))

(1555345, 26)

In [11]:
display(df.select('reviewYear').groupBy('reviewYear').count().orderBy('ReviewYear', ascending=True))

reviewYear,count
2011,61439
2012,46533
2013,135359
2014,240560
2015,308759
2016,398895
2017,363800


In [12]:
df.printSchema()

root
-- reviewID: long (nullable = true)
-- overall: double (nullable = true)
-- vote: integer (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: date (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- label: integer (nullable = true)
-- lastDate: date (nullable = false)
-- reviewAge: integer (nullable = true)
-- reviewYear: integer (nullable = true)
-- Y2011: integer (nullable = false)
-- Y2012: integer (nullable = false)
-- Y2013: integer (nullable = false)
-- Y2014: integer (nullable = false)
-- Y2015: integer (nullable = false)
-- Y2016: integer (nullable = false)
-- Y2017: integer (nullable = false)
-- Overall_1: integer (nullable = false)
-- Overall_2: integer (nullable = false)
-- Overall_3: integer (nullable = false)
-- Overall_4: integer (nullable = false)
-- Overall_5: integer (nullable = false)

In [13]:
# For our intitial modeling efforts, we are not going to use the following features
drop_list = ['asin', 'reviewID', 'reviewerID','reviewTime', 'image', 'style', 'reviewerName']
df = df.select([column for column in df.columns if column not in drop_list])
df = df.na.drop(subset=["reviewText", "label"])
df.show(5)
print((df.count(), len(df.columns)))

+-------+----+--------+--------------------+--------------------+-----+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+
overall|vote|verified| reviewText| summary|label| lastDate|reviewAge|reviewYear|Y2011|Y2012|Y2013|Y2014|Y2015|Y2016|Y2017|Overall_1|Overall_2|Overall_3|Overall_4|Overall_5|
+-------+----+--------+--------------------+--------------------+-----+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+
 5.0| 0| true|This game is a bi...|but when you do i...| 0|2017-12-31| 806| 2015| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1|
 4.0| 0| false|I played it a whi...|But in spite of t...| 0|2017-12-31| 888| 2015| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0|
 5.0| 0| true|I bought this gam...|A very good game ...| 0|2017-12-31| 1399| 2014| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1|
 5.0| 0| true|I have played the...|Anno 2070 more li...| 0|2017-12-31| 1409| 2014| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1|
 4.0| 0| true|4 Stars because t...|My boys enjoys th...| 0|2017-12-31| 1829| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0|
+-------+----+--------+--------------------+--------------------+-----+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+
only showing top 5 rows

(1555345, 21)

In [14]:
#https://stackoverflow.com/questions/51450004/spark-dataframe-python-count-substring-in-string
#There are some cases where there are more spaces than characters. These give negative numbers. We will replace with 0
df = df.withColumn('word_count', f.size(f.split(f.col('reviewText'), ' ')))
df = df.withColumn('character_count',  when((f.length(trim(col('reviewText'))) - size(split(trim(col("reviewText")), " ")) - 1)<0,0).otherwise(f.length(trim(col('reviewText'))) - size(split(trim(col("reviewText")), " ")) - 1))
df = df.withColumn('sentence_count', f.size(f.split(f.col('reviewText'), r"\.")))
df = df.withColumn('avg_word_length', f.col('character_count')/f.col('word_count'))
df = df.withColumn('avg_sentence_length', f.col('word_count')/f.col('sentence_count'))

In [15]:
df = df.withColumn('sum_word_count', f.size(f.split(f.col('summary'), ' ')))
df = df.withColumn('sum_character_count',  when((f.length(trim(col('summary'))) - size(split(trim(col("summary")), " ")) - 1)<0,0).otherwise(f.length(trim(col('summary'))) - size(split(trim(col("summary")), " ")) - 1))
df = df.fillna(0, subset=['sum_character_count'])
df = df.withColumn('sum_sentence_count', f.size(f.split(f.col('summary'), r"\.")))
df = df.withColumn('sum_avg_word_length', f.col('character_count')/f.col('word_count'))
df = df.withColumn('sum_avg_sentence_length', f.col('word_count')/f.col('sentence_count'))

In [16]:
from textblob import TextBlob
def sentiment_score(chat):
    return TextBlob(chat).sentiment.polarity
  
from pyspark.sql.types import FloatType
sentiment_score_udf = f.udf(lambda x: sentiment_score(x), FloatType())

In [17]:
df = df.withColumn('sentiment_score',sentiment_score_udf('reviewText'))
test_df = test_df.withColumn('sentiment_score',sentiment_score_udf('reviewText'))

In [18]:
from pyspark.sql.types import FloatType

# Re-balancing (weighting) of records to be used in the logistic loss objective function
numPositives = df.filter(df["label"] == 1).count()
datasetSize = df.count()
balancingRatio = (datasetSize - numPositives) / datasetSize
print("numPositives   = {}".format(numPositives))
print("datasetSize    = {}".format(datasetSize))
print("balancingRatio = {}".format(balancingRatio))

def calculateWeights(d, multiplier = 1):
    if d == 1.0:
      return multiplier * balancingRatio
    else:
      return (1.0 - balancingRatio)
    
udfcalculateWeights = f.udf(calculateWeights, FloatType())
    
df = df.withColumn("classWeightCol", udfcalculateWeights(df["label"]))

numPositives = 180220
datasetSize = 1555345
balancingRatio = 0.884128601692872

In [19]:
#http://michael-harmon.com/blog/SentimentAnalysisP2.html#bullet5
from pyspark import keyword_only
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
import nltk
from nltk.stem.porter import PorterStemmer


class PorterStemming(Transformer, HasInputCol, HasOutputCol):
    """
    PosterStemming class using the NLTK Porter Stemmer
    
    This comes from https://stackoverflow.com/questions/32331848/create-a-custom-transformer-in-pyspark-ml
    Adapted to work with the Porter Stemmer from NLTK.
    """
    
    @keyword_only
    def __init__(self, 
                 inputCol  : str = None, 
                 outputCol : str = None, 
                 min_size  : int = None):
        """
        Constructor takes in the input column name, output column name,
        plus the minimum legnth of a token (min_size)
        """
        # call Transformer classes constructor since were extending it.
        super(Transformer, self).__init__()

        # set Parameter objects minimum token size
        self.min_size = Param(self, "min_size", "")
        self._setDefault(min_size=0)

        # set the input keywork arguments
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

        # initialize Stemmer object
        self.stemmer  = PorterStemmer()

        
    @keyword_only
    def setParams(self, 
                  inputCol  : str = None, 
                  outputCol : str = None, 
                  min_size  : int = None
      ) -> None:
        """
        Function to set the keyword arguemnts
        """
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    

    def _stem_func(self, words  : list) -> list:
        """
        Stemmer function call that performs stemming on a
        list of tokens in words and returns a list of tokens
        that have meet the minimum length requiremnt.
        """
        # We need a way to get min_size and cannot access it 
        # with self.min_size
        min_size       = self.getMinSize()

        # stem that actual tokens by applying 
        # self.stemmer.stem function to each token in 
        # the words list
        stemmed_words  = map(self.stemmer.stem, words)

        # now create the new list of tokens from
        # stemmed_words by filtering out those
        # that are not of legnth > min_size
        filtered_words = filter(lambda x: len(x) > min_size, stemmed_words)

        return list(filtered_words)
    
    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transform function is the method that is called in the 
        MLPipleline.  We have to override this function for our own use
        and have it call the _stem_func.

        Notice how it takes in a type DataFrame and returns type Dataframe
        """
        # Get the names of the input and output columns to use
        out_col       = self.getOutputCol()
        in_col        = self.getInputCol()

        # create the stemming function UDF by wrapping the stemmer 
        # method function
        stem_func_udf = F.udf(self._stem_func, ArrayType(StringType()))
        
        # now apply that UDF to the column in the dataframe to return
        # a new column that has the same list of words after being stemmed
        df2           = df.withColumn(out_col, stem_func_udf(df[in_col]))

        return df2
  
  
    def setMinSize(self,value):
        """
        This method sets the minimum size value
        for the _paramMap dictionary.
        """
        self._paramMap[self.min_size] = value
        return self

    def getMinSize(self) -> int:
        """
        This method uses the parent classes (Transformer)
        .getOrDefault method to get the minimum
        size of a token.
        """
        return self.getOrDefault(self.min_size)


In [20]:
df.show(5)

+-------+----+--------+--------------------+--------------------+-----+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+----------+---------------+--------------+------------------+-------------------+--------------+-------------------+------------------+-------------------+-----------------------+---------------+--------------+
overall|vote|verified| reviewText| summary|label| lastDate|reviewAge|reviewYear|Y2011|Y2012|Y2013|Y2014|Y2015|Y2016|Y2017|Overall_1|Overall_2|Overall_3|Overall_4|Overall_5|word_count|character_count|sentence_count| avg_word_length|avg_sentence_length|sum_word_count|sum_character_count|sum_sentence_count|sum_avg_word_length|sum_avg_sentence_length|sentiment_score|classWeightCol|
+-------+----+--------+--------------------+--------------------+-----+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+----------+---------------+--------------+------------------+-------------------+--------------+-------------------+------------------+-------------------+-----------------------+---------------+--------------+
 5.0| 0| true|This game is a bi...|but when you do i...| 0|2017-12-31| 806| 2015| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 17| 53| 2|3.1176470588235294| 8.5| 6| 20| 2| 3.1176470588235294| 8.5| 0.036111113| 0.1158714|
 4.0| 0| false|I played it a whi...|But in spite of t...| 0|2017-12-31| 888| 2015| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 66| 230| 6| 3.484848484848485| 11.0| 11| 31| 1| 3.484848484848485| 11.0| 0.08981481| 0.1158714|
 5.0| 0| true|I bought this gam...|A very good game ...| 0|2017-12-31| 1399| 2014| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 148| 560| 6|3.7837837837837838| 24.666666666666668| 11| 43| 1| 3.7837837837837838| 24.666666666666668| -0.10854342| 0.1158714|
 5.0| 0| true|I have played the...|Anno 2070 more li...| 0|2017-12-31| 1409| 2014| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 172| 718| 11| 4.174418604651163| 15.636363636363637| 6| 22| 1| 4.174418604651163| 15.636363636363637| 0.031944446| 0.1158714|
 4.0| 0| true|4 Stars because t...|My boys enjoys th...| 0|2017-12-31| 1829| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 30| 117| 3| 3.9| 10.0| 6| 28| 1| 3.9| 10.0| -0.108333334| 0.1158714|
+-------+----+--------+--------------------+--------------------+-----+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+----------+---------------+--------------+------------------+-------------------+--------------+-------------------+------------------+-------------------+-----------------------+---------------+--------------+
only showing top 5 rows

In [21]:
# In Spark's MLLib, it's considered good practice to combine all the preprocessing steps into a pipeline.
# That way, you can run the same steps on both the training data, and testing data and beyond (new data)
# without copying and pasting any code.

# It is possible to run all of these steps one-by-one, outside of a Pipeline, if desired. But that's
# not how I am going to do it here.

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, VectorAssembler, IDF, NGram
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.ml.classification import GBTClassifier
#https://medium.com/@aieeshashafique/gradient-boost-model-using-pyspark-mllib-solving-a-chronic-kidney-disease-problem-13039b6dc099
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import OneHotEncoderEstimator

# We'll tokenize the text using a simple RegexTokenizer
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")

# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

# TODO: insert other clearning steps here (and put into the pipeline, of course!)
# E.g., n-grams? document length?


#stem = PorterStemming(inputCol="filtered", outputCol="stemmed")
bigram = NGram(inputCol="filtered", outputCol="bigrams", n=2)
#assembler_ngram = VectorAssembler(inputCols=['filtered', 'bigrams'], outputCol='combined')

# Vectorize the sentences using simple BOW method. Other methods are possible:
# https://spark.apache.org/docs/2.2.0/ml-features.html#feature-extractors

tf = CountVectorizer(inputCol="bigrams", outputCol="rawFeatures", vocabSize=2000, minTF=1, maxDF=0.40)
# Generate Inverse Document Frequency weighting
idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures", minDocFreq=100)

#tokenizer_sum = RegexTokenizer(inputCol="summary", outputCol="words_sum", pattern="\\W")
#stopwordsRemover_sum = StopWordsRemover(inputCol="words_sum", outputCol="filtered_sum")
#bigram_sum = NGram(inputCol="filtered_sum", outputCol="bigrams_sum", n=2)
#tf_sum = CountVectorizer(inputCol="bigrams_sum", outputCol="rawFeatures_sum", vocabSize=2000, minTF=1, maxDF=0.40)
#idf_sum = IDF(inputCol="rawFeatures_sum", outputCol="idfFeatures_sum", minDocFreq=100)

#one-hot-encoder
#encode_overall = OneHotEncoder(inputCol="overall", outputCol="overall_ohe")
#encode_year = OneHotEncoderEstimator(inputCols=["reviewYear"], outputCols=["year_ohe"], handleInvalid="keep")
# Combine all features into one final "features" column
#assembler = VectorAssembler(inputCols=['character_count','word_count','sentence_count','avg_word_length','avg_sentence_length', 'reviewAge', "verified", "Y2011", "Y2012", "Y2013", "Y2014", "Y2015", "Y2016", "Y2017", "Y2018",
#                                       "Overall_1", "Overall_2", "Overall_3", "Overall_4", "Overall_5", "idfFeatures", "idfFeatures_sum"], outputCol="features")
#assembler = VectorAssembler(inputCols=['character_count','word_count','sentence_count','avg_word_length','avg_sentence_length', 'reviewAge', "verified", "Y2011", "Y2012", "Y2013", "Y2014", "Y2015", "Y2016", "Y2017", "Y2018",
#                                       "Overall_1", "Overall_2", "Overall_3", "Overall_4", "Overall_5", "idfFeatures"], outputCol="features")
assembler = VectorAssembler(inputCols=['character_count','word_count','sentence_count','avg_word_length','avg_sentence_length', 'reviewAge', "verified", "Y2011", "Y2012", "Y2013", "Y2014", "Y2015", "Y2016", "Y2017",
                                       "Overall_1", "Overall_2", "Overall_3", "Overall_4", "Overall_5", "sentiment_score", 'sum_word_count', 'sum_character_count', 'sum_sentence_count', 'sum_avg_word_length', 'sum_avg_sentence_length', "idfFeatures"], outputCol="features")

# Machine Learning Algorithm

#ml_alg  = LogisticRegression(maxIter=10, weightCol="classWeightCol")
#ml_alg  = NaiveBayes(modelType="multinomial", weightCol="classWeightCol")
ml_alg = GBTClassifier()
#ml_alg  = RandomForestClassifier(numTrees=100, featureSubsetStrategy="auto", impurity='gini', maxDepth=4, maxBins=32)


#pipeline = Pipeline(stages=[tokenizer, stopwordsRemover, stem, bigram, tf, idf, encoder, assembler, ml_alg])
pipeline = Pipeline(stages=[tokenizer, stopwordsRemover, bigram, tf, idf, assembler, ml_alg])

#paramGrid = ParamGridBuilder() \
#    .addGrid(ml_alg.regParam, [0.0, 0.19]) \
#    .addGrid(ml_alg.elasticNetParam, [0.0, 0.5]) \
#    .build()

#paramGrid = ParamGridBuilder() \
#    .addGrid(ml_alg.smoothing, [0.0, 0.5, 1.0, 1.5]) \
#    .build()

paramGrid = ParamGridBuilder() \
    .addGrid(ml_alg.maxIter, [30,40]) \
    .build()

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)  # use 3+ folds in practice

In [22]:
# set seed for reproducibility
(trainingData, testData) = df.randomSplit([0.9, 0.1], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count:     " + str(testData.count()))

Training Dataset Count: 1399978
Test Dataset Count: 155367

In [23]:
pipelineFit = crossval.fit(trainingData)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [24]:
predictions =  pipelineFit.transform(testData)
predictions.groupBy("prediction").count().show()

+----------+------+
prediction| count|
+----------+------+
 0.0|148193|
 1.0| 7174|
+----------+------+

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
pre_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
rec_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
pr_evaluator  = BinaryClassificationEvaluator(metricName="areaUnderPR")
auc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

print("Test Accuracy       = %g" % (acc_evaluator.evaluate(predictions)))
print("Test Precision      = %g" % (pre_evaluator.evaluate(predictions)))
print("Test Recall         = %g" % (rec_evaluator.evaluate(predictions)))
print("Test areaUnderPR    = %g" % (pr_evaluator.evaluate(predictions)))
print("Test areaUnderROC   = %g" % (auc_evaluator.evaluate(predictions)))

Test Accuracy = 0.896374
Test Precision = 0.87753
Test Recall = 0.896374
Test areaUnderPR = 0.492858
Test areaUnderROC = 0.859349

In [26]:
# Load in the tables

test_df.show(5)
print((test_df.count(), len(test_df.columns)))
test_df.describe().show()

+--------+-------+--------+----------+--------------+----------+----------------+--------------------+--------------------+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------------+
reviewID|overall|verified|reviewTime| reviewerID| asin| reviewerName| reviewText| summary| lastDate|reviewAge|reviewYear|Y2011|Y2012|Y2013|Y2014|Y2015|Y2016|Y2017|Overall_1|Overall_2|Overall_3|Overall_4|Overall_5|sentiment_score|
+--------+-------+--------+----------+--------------+----------+----------------+--------------------+--------------------+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------------+
67000000| 3| true|2012-05-16|A3IXM075VM1P9T|B007JYB3O2| nachtik|I would say these...| average reading|2017-12-31| 2055| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0.2595238|
67000001| 5| true|2012-12-25|A3LGZ7A3WSV3JJ| 985719745| MSP|WOW, DROPPIN DIME...|WOW.................|2017-12-31| 1832| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0.093333334|
67000002| 5| true|2012-09-18|A3BF5G7CJNIAG0|B002KXH7PQ| C. S. DeMore|You'll love the p...|Another Buggy Bar...|2017-12-31| 1930| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0.309375|
67000003| 5| false|2015-07-27|A2W41RTHSHYC4Q|B011LXVWRO| Light|For the majority ...|This is a complet...|2017-12-31| 888| 2015| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0.32205883|
67000004| 5| false|2003-03-29| ACTBQZV1CJ9E8|9706061681|Richard Eastwood|MI MEJOR AMIGO..Q...|LE ACABA DE SUCED...|2017-12-31| 5391| 2003| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| -0.4375|
+--------+-------+--------+----------+--------------+----------+----------------+--------------------+--------------------+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------------+
only showing top 5 rows

(304984, 25)
+-------+----------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-------------------+------------------+-------------------+
summary| reviewID| overall| reviewerID| asin| reviewerName| reviewText| summary| reviewAge| reviewYear| Y2011| Y2012| Y2013| Y2014| Y2015| Y2016| Y2017| Overall_1| Overall_2| Overall_3| Overall_4| Overall_5| sentiment_score|
+-------+----------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-------------------+------------------+-------------------+
 count| 304984| 304984| 304984| 304984| 304846| 304984| 304855| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984| 304984|
 mean| 6.71524915E7| 4.32360058232563| null|2.3012381768059964E9|1.2594881926923077E8|1.541666666666666...|14364.488500000001|887.4275273456968|2015.0393528840857|0.039044671195866014|0.029873698292369435|0.08749311439288618|0.15473598615009312|0.1983677832279726|0.2566823177609317|0.2338024289798809|0.039907011515358186|0.03981192455997692|0.0897653647404454|0.21780486845211552| 0.612710830732104| 0.2758891461719976|
 stddev|88041.4415867134|1.0564910412520399| null|2.1914585491320453E9| 9.906705755265955E8|3.776296686790733E21| 90140.69312527789|748.4511948178172|2.0293887473522956| 0.1937015949106458| 0.1702391126284671|0.28255677501251397| 0.3616534108685826| 0.398771271789687|0.4368033094048113|0.4232486746065875| 0.1957410727859513| 0.195

In [27]:
test_df = test_df.withColumn('word_count', f.size(f.split(f.col('reviewText'), ' ')))
test_df = test_df.withColumn('character_count',  when((f.length(trim(col('reviewText'))) - size(split(trim(col("reviewText")), " ")) - 1)<0,0).otherwise(f.length(trim(col('reviewText'))) - size(split(trim(col("reviewText")), " ")) - 1))
test_df = test_df.withColumn('sentence_count', f.size(f.split(f.col('reviewText'), r"\.")))
test_df = test_df.withColumn('avg_word_length', f.col('character_count')/f.col('word_count'))
test_df = test_df.withColumn('avg_sentence_length', f.col('word_count')/f.col('sentence_count'))



In [28]:
test_df = test_df.withColumn('sum_word_count', f.size(f.split(F.col('summary'), ' ')))
test_df = test_df.withColumn('sum_character_count',  when((f.length(trim(col('summary'))) - size(split(trim(col("summary")), " ")) - 1)<0,0).otherwise(f.length(trim(col('summary'))) - size(split(trim(col("summary")), " ")) - 1))
test_df = test_df.fillna(0, subset=['sum_character_count'])
test_df = test_df.withColumn('sum_sentence_count', f.size(f.split(F.col('summary'), r"\.")))
test_df = test_df.withColumn('sum_avg_word_length', f.col('character_count')/f.col('word_count'))
test_df = test_df.withColumn('sum_avg_sentence_length', f.col('word_count')/f.col('sentence_count'))

In [29]:
kaggle_pred = pipelineFit.transform(test_df)
kaggle_pred.show(5)

+--------+-------+--------+----------+--------------+----------+----------------+--------------------+--------------------+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------------+----------+---------------+--------------+-----------------+-------------------+--------------+-------------------+------------------+-------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
reviewID|overall|verified|reviewTime| reviewerID| asin| reviewerName| reviewText| summary| lastDate|reviewAge|reviewYear|Y2011|Y2012|Y2013|Y2014|Y2015|Y2016|Y2017|Overall_1|Overall_2|Overall_3|Overall_4|Overall_5|sentiment_score|word_count|character_count|sentence_count| avg_word_length|avg_sentence_length|sum_word_count|sum_character_count|sum_sentence_count|sum_avg_word_length|sum_avg_sentence_length| words| filtered| bigrams| rawFeatures| idfFeatures| features| rawPrediction| probability|prediction|
+--------+-------+--------+----------+--------------+----------+----------------+--------------------+--------------------+----------+---------+----------+-----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------------+----------+---------------+--------------+-----------------+-------------------+--------------+-------------------+------------------+-------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
67000000| 3| true|2012-05-16|A3IXM075VM1P9T|B007JYB3O2| nachtik|I would say these...| average reading|2017-12-31| 2055| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0.2595238| 47| 213| 4|4.531914893617022| 11.75| 2| 12| 1| 4.531914893617022| 11.75|[i, would, say, t...|[say, 5, regular,...|[say 5, 5 regular...|(2000,[11,820,151...|(2000,[11,820,151...|(2025,[0,1,2,3,4,...|[0.64931120041034...|[0.78560304406352...| 0.0|
67000001| 5| true|2012-12-25|A3LGZ7A3WSV3JJ| 985719745| MSP|WOW, DROPPIN DIME...|WOW.................|2017-12-31| 1832| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0.093333334| 132| 596| 21|4.515151515151516| 6.285714285714286| 1| 18| 18| 4.515151515151516| 6.285714285714286|[wow, droppin, di...|[wow, droppin, di...|[wow droppin, dro...| (2000,[1193],[1.0])|(2000,[1193],[7.2...|(2025,[0,1,2,3,4,...|[0.25307548276867...|[0.62390373839294...| 0.0|
67000002| 5| true|2012-09-18|A3BF5G7CJNIAG0|B002KXH7PQ| C. S. DeMore|You'll love the p...|Another Buggy Bar...|2017-12-31| 1930| 2012| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0.309375| 44| 189| 2|4.295454545454546| 22.0| 4| 21| 1| 4.295454545454546| 22.0|[you, ll, love, t...|[ll, love, patter...|[ll love, love pa...|(2000,[74,146,571...|(2000,[74,146,571...|(2025,[0,1,2,3,4,...|[0.80690692698122...|[0.83394021679277...| 0.0|
67000003| 5| false|2015-07-27|A2W41RTHSHYC4Q|B011LXVWRO| Light|For the majority ...|This is a complet...|2017-12-31| 888| 2015| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0.32205883| 160| 736| 10| 4.6| 16.0| 8| 35| 1| 4.6| 16.0|[for, the, majori...|[majority, person...|[majority persons...|(2000,[287,670],[...|(2000,[287,670],[...|(2025,[0,1,2,3,4,...|[0.50098717744933...|[0.73144658323304...| 0.0|
67000004| 5| false|2003-03-29| ACTBQZV1CJ9E8|9706061681|Richard Eastwood|MI MEJOR AMIGO..Q...|LE ACABA DE SUCED...|2017-12-31| 5391| 2003| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| -0.4375| 84| 363| 8|4.321428571428571| 10.5| 9| 28| 1| 4.321428571428571| 10.5|[mi, mejor, amigo...|[mi, mejor, amigo...|[mi mejor, mejor ...| (2000,[],[])| (2000,[],[])|(2025,[0,1,2,3,4,...|[-0.0927237791136...|[0.45377052360949...| 1.0|
+--------+-------+--------+----------+--------------+----------+----------------+--------------------+--------------------+----------+---------+-----

In [30]:
#kaggle_pred.printSchema()

In [31]:
kaggle_pred.groupBy("prediction").count().show()

+----------+------+
prediction| count|
+----------+------+
 0.0|289861|
 1.0| 15123|
+----------+------+

In [32]:
#display(kaggle_pred.select(["reviewID", "prediction"]))

In [33]:
secondelement=udf(lambda v:float(v[1]),FloatType())
#https://stackoverflow.com/questions/44425159/access-element-of-a-vector-in-a-spark-dataframe-logistic-regression-probability

In [34]:
#kaggle_pred.select(secondelement('probability')).show(5)

In [35]:
display(kaggle_pred.select(["reviewID", secondelement('probability')]))

reviewID,(probability)
67000000,0.21439695
67000001,0.37609625
67000002,0.16605978
67000003,0.2685534
67000004,0.5462295
67000005,0.17506598
67000006,0.29141527
67000007,0.37609625
67000008,0.8107678
67000009,0.37403417
